In [ ]:
import pandas as pd

data = pd.read_excel("Test_Plan.xlsx")
df = pd.DataFrame(data)

def Overlap_Checker(df):
    """
    Checks if the in the current busplan if there are overlapping trips for each bus. 
    :param df: The given dataframe
    """

    def check_overlaps(group):
        """
        Function to check overlapping time in trips for a single bus. 
        :param group: Group of trips for a single bus
        :return: List of overlapping trips (with row numbers)
        """
        overlaps = []
        # .groupby resets the index, this prevents the reset of the original index when sorting by bus later in the code.
        group = group.reset_index(drop=False)
        
        for i in range(len(group)):
            for j in range(i + 1, len(group)):
                # It is an overlap if the end time of a trip is after the start time of a trip after the first.     
                # Start time of a trip should be before the end of a trip after the first, so that there is no overlap with midnight trips.
                if (group.loc[i, 'start time'] < group.loc[j, 'end time'] and
                    group.loc[j, 'start time'] < group.loc[i, 'end time']):
                    #appends in a in a tuple, so that there is a list with tuples of the data of the overlapping trips.
                    overlaps.append((
                        group.loc[i, 'index'], 
                        group.loc[j, 'index'], 
                        group.loc[i, 'start location'], group.loc[i, 'end location'], 
                        group.loc[i, 'start time'], group.loc[i, 'end time'],
                        group.loc[j, 'start location'], group.loc[j, 'end location'], 
                        group.loc[j, 'start time'], group.loc[j, 'end time'],
                    ))
        return overlaps

    # Group by bus and run the check_overlaps function. 
    overlap_results = []
    grouped = df.groupby('bus')
    # For each bus, repeats the amount that the group is long.
    for bus, group in grouped:
        overlaps = check_overlaps(group)
        for o in overlaps:
            # At the start of each tuple, append the number of the bus of that group.
            # So that later when printing the results, it is possible to state wich bus has overlap.
            overlap_results.append((bus, *o))



    for o in overlap_results:
        # For each tuple in overlap_results, the first character is the bus number.
        bus = o[0]
        print(f"Bus {bus} has overlap:")
        print(f"  row {o[1]}: {o[3]} -> {o[4]} ({o[5]} - {o[6]})")
        print(f"  row {o[2]}: {o[7]} -> {o[8]} ({o[9]} - {o[10]})")

Overlap_Checker(df)


Bus 1 has overlap:
  row 8: ehvbst -> ehvbst (07:21:00 - 08:31:00)
  row 9: ehvbst -> ehvapt (07:21:00 - 07:45:00)
Bus 1 has overlap:
  row 8: ehvbst -> ehvbst (07:21:00 - 08:31:00)
  row 10: ehvapt -> ehvapt (07:45:00 - 07:46:00)
Bus 1 has overlap:
  row 8: ehvbst -> ehvbst (07:21:00 - 08:31:00)
  row 11: ehvapt -> ehvbst (07:46:00 - 08:11:00)
Bus 1 has overlap:
  row 8: ehvbst -> ehvbst (07:21:00 - 08:31:00)
  row 12: ehvbst -> ehvapt (08:11:00 - 08:31:00)
